In [1]:
!pip3 install kaggle
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(
q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "content/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
  status, done = downloader.next_chunk()
  print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)


!mkdir ~/.kaggle
!cp /content/content/kaggle.json ~/.kaggle/kaggle.json

!mkdir data
!kaggle competitions download -c retail-case-study-b7 -p data

Download 100%.
  0% 0.00/329k [00:00<?, ?B/s]
100% 329k/329k [00:00<00:00, 43.8MB/s]
  0% 0.00/5.31k [00:00<?, ?B/s]
100% 5.31k/5.31k [00:00<00:00, 4.42MB/s]
  0% 0.00/793 [00:00<?, ?B/s]
100% 793/793 [00:00<00:00, 584kB/s]
  0% 0.00/309 [00:00<?, ?B/s]
100% 309/309 [00:00<00:00, 283kB/s]
  0% 0.00/17.4k [00:00<?, ?B/s]
100% 17.4k/17.4k [00:00<00:00, 15.4MB/s]
  0% 0.00/12.4k [00:00<?, ?B/s]
100% 12.4k/12.4k [00:00<00:00, 12.0MB/s]
  0% 0.00/10.2k [00:00<?, ?B/s]
100% 10.2k/10.2k [00:00<00:00, 10.8MB/s]


In [3]:
!mkdir ../input/
!kaggle competitions download -c retail-case-study-b7 -p ../input/
!ls data

mkdir: cannot create directory ‘../input/’: File exists
WeatherData.xlsx: Skipping, found more recently modified local copy (use --force to force download)
Train_Kaggle.csv: Skipping, found more recently modified local copy (use --force to force download)
Test_Kaggle.csv: Skipping, found more recently modified local copy (use --force to force download)
Sample_Submission.csv: Skipping, found more recently modified local copy (use --force to force download)
MacroEconomicData.xlsx: Skipping, found more recently modified local copy (use --force to force download)
Events_HolidaysData.xlsx: Skipping, found more recently modified local copy (use --force to force download)
AttributesDescription.xlsx: Skipping, found more recently modified local copy (use --force to force download)
AttributesDescription.xlsx  Sample_Submission.csv  WeatherData.xlsx
Events_HolidaysData.xlsx    Test_Kaggle.csv
MacroEconomicData.xlsx	    Train_Kaggle.csv


In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

%matplotlib inline

import math, time, random, datetime

# Data Manipulation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Visualisation
import matplotlib.pyplot as plt
import missingno
import seaborn as sns
plt.style.use('seaborn-whitegrid')

# Preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier


## Importing required packages
import string
import random
import re
import torch
import torch.nn as nn
from torch.autograd import Variable
import os


# Lets ignore the warnings for now
import warnings
warnings.filterwarnings('ignore')

# for RNN
import string
import random
import re
import torch
import torch.nn as nn
from torch.autograd import Variable

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

print(os.listdir("../input"))

trainPD = pd.read_csv('../input/Train_Kaggle.csv')
testPD  = pd.read_csv('../input/Test_Kaggle.csv')
sampleSubPD  = pd.read_csv('../input/Sample_Submission.csv')

attDescPD = pd.read_excel('../input/AttributesDescription.xlsx')
eventHolidaysPD = pd.read_excel('../input/Events_HolidaysData.xlsx')
macroEconomicsPD = pd.read_excel('../input/MacroEconomicData.xlsx')

['Train_Kaggle.csv', 'Test_Kaggle.csv', 'Events_HolidaysData.xlsx', 'WeatherData.xlsx', 'Sample_Submission.csv', 'MacroEconomicData.xlsx', 'AttributesDescription.xlsx']


In [0]:
### Creating recurrent neural network
class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [0]:
def train(inp, target):
    hidden = decoder.init_hidden()
    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = decoder(inp[c], hidden)
        '''unsqueeze() is used to add dimension to the tensor'''
        loss += criterion(output, target[c].unsqueeze(dim=0))
    # Back propagation
    loss.backward()
    decoder_optimizer.step()

    return loss.item() / chunk_len

In [0]:
all_characters = string.printable
# Turn string into list of longs
def char_tensor(string):
    ##  tensor is a array
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)

print(char_tensor('abcDEF'))

In [0]:
def get_training_set(trainPD):    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [0]:
n_epochs = 2000 #Number of epochs
print_every = 50
plot_every = 20
hidden_size = 100
n_layers = 1
lr = 0.005

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
## Optimizer
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
## Loss function
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    loss = train(*get_training_set())       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(evaluate('Wh', 100), '\n')

    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

In [0]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

plt.figure()
plt.plot(all_losses)
plt.xlabel("epoch")
plt.ylabel("loss")